<a href="https://colab.research.google.com/github/timcowlishaw/gpt-2/blob/finetuning/Making_talking_robots_with_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We're going to train a GPT-2 Model!

GPT-2 is the [state of the art text generation model](https://openai.com/blog/better-language-models/) developed by the OpenAI foundation. They only released a limited, smaller version of their model due to concerns about it being misused to generate fake news and the like. **Is this something that worries you? At the end of this tutorial, have a think about whether you feel the same, or differently.**

One of the cool things about GPT-2 (and machine learning approaches that use _Deep Neural Networks_ in general) is that we can _fine-tune_ them - this means we can take an existing model (such as GPT-2's model of English text on the internet), and use them as a starting point to learn something more specific. 

This is super useful, as it takes a *lot* of data to learn a model of language. If we wanted to, for instance, make a model that speaks like [Columbo](https://en.wikipedia.org/wiki/Columbo), there simply isn't enough example text from columbo scripts to do that from scratch (for some idea of the amount of data you need - there probably isn't enough data in the complete works of Shakespeare to train a 'shakespeare model', either). 

However, if we start from the pre-trained GPT-2 model, we can take advantage of the fact that GPT-2 has already learned a pretty usable representation of the English lanaguage from gigabytes and gigabytes of text scraped from the internet, and we can build _on top of it_ use a _smaller_ amount of example text in the style we want, in order to learn to generate text in that style. This is what _fine-tuning_ is, and we're going to try it out today! We're going to follow the process in [this technical blog post](https://medium.com/@ngwaifoong92/beginners-guide-to-retrain-gpt-2-117m-to-generate-custom-text-content-8bb5363d8b7f), but without needing to write any code or install any software ourseleves.

## Things you'll need:

 - A [Google Drive](https://www.google.com/drive/) account
 - A plain text file containing the text you want to train from. This could be the text of a novel, some quotations from a favourite TV or film character, lyrics from a favourite singer or band, or your own WhatsApp or iChat logs. This is your _training set_. 
  - If this isn't text you've written yourself, please be mindful and respectful of whoever wrote it, (For starters, you should ask their permission, and never use the output of this model to impersonate them for the purposes of deception), as well as respecting any relevant laws in your local area, such as copyright. **In general, if you're planning to use your model to insult, defame, deceive or otherwise harm someone in any way, then stop now and have a good think about what you're doing. This tutorial is _not_ for you.**
  - Each individual _example_ (an individual message, quotation, paragraph or location) should be on its own line). 
  - If it makes sense to 'group' these examples in some way (for instance, if they're song lyrics, each example might be a _line_, which would be naturally grouped into _songs_, with your whole _training set_ being the complete works given artist, then you can optionally separate the groups with this special word: `<|endoftext|>`. Copy this (including the punctuation) onto its own line in between each group in your training set. This helps the model understand the structure of your examples better and hence generate a better simulation!
   - Once you've got this file ready, upload it to your google drive account and make a note of the filename, which we'll need in a second.

## OK, off we go!

This file is a _Google Colab notebook_. It's a special document which mixes normal prose (for humans) and computer code. We can read it like a document, but also run it, like a computer program. It's made up of _cells_ (the grey boxes below), each of which contains a bit of computer code which performs a particular step of our process. We're going to go through the cells in order, and _execute_ them, and by the end, you'll have trained your own machine learning text model, and you'll not have to write a single line of computer code, and hopefully learned a little bit about some of the principles of machine learning along the way. 

This text will provide explanation, commentary and context for what you're doing. From now on, any instructions for things you need to do will be given in **bold type**.

First, it'd be a good idea to make a copy of this notebook, so that you've got one of your own to keep! **Click the _File_ menu above, and choose _'Save a copy in Drive'_**. This will create a new copy in your own Google Drive. **Go to Google Drive and  open it in a new tab** - we'll see you over there!

## Hello again :-)

Right, on we go!

Let's get back to those _cells_ we were talking about before (the grey areas of the computer document containing computer code you can see below). **To execute a cell, click your mouse inside the grey area, and you'll see a little black 'play' icon pop up to the left. You can then click that icon to execute the cell.** The play icon will animate to show that the computer is 'thinking'. When it's finished, the animation will stop, and you can move on to the next step. Occasionally, a cell will print out text as it executes, which gives you more information about what it's doing. It might also sometimes prompt you for some more information. We'll warn you if you need to pay attention to these!

If you lose your place - look at the left hand side of the cells. Cells beginning with `[ ]` have not been executed yet, and those with a number like `[2]` have already been executed. The number indicates the order in which they were executed. Find the first line without a number and start there!

We'll give you notes on what's happening as we go along, as well as further instructions if necessary (remember, things *you* need to do will be **in bold**). If you want more detail, look for the green lines beginning with a '#' inside the cells, among the computer code. These are _comments_ - lines of plain english which the computer ignores, but which we use to explain to anyone reading what the code we've written does.

**First up - we need to tell the computer know where it can find the _training set_ we created and uploaded to Google Drive earlier. Add that in the form below, then execute the cell to the left by hitting the play button on the left to move onto the next step**! Note also how the code changes as you change the filename in the form.

# The code starts here:



In [0]:
#@title Training data file name
training_file_name="catalonia.txt" #@param{type: 'string'}

We're on our way! The fact that this is going to be so easy is due to the fact that we can re-use code written by other programmers and machine learning researchers, so we don't have to do all the work ourselves. We'll start by fetching some of that code so we can use it, and that's what the next few cells do. Programmers often use a service called [_Github_](https://github.com) to store and share their code, this makes it easy to store the code we've written and share it with others. Our first command will fetch the gpt-2 code from github, then the next few cells set up the notebook to use it properly. **Execute these in order now**:


In [0]:
%%capture
# Download the gpt-2 fine tuning code from github.
!test -d gpt-2 || git clone https://github.com/timcowlishaw/gpt-2.git

In [0]:
%%capture
# Change our working directory to the gpt-2 directory we just downloaded.
import os
if not os.getcwd().endswith("gpt-2"):
  os.chdir("gpt-2")
# If we're starting again after partially working our way through the notebook
# make sure the code's up to date first.
!git pull --rebase

In [0]:
%%capture
# Make sure all the gpt-2 library code is loadable in python.
import sys
sys.path.append("./src")

Even programmers rely on code written by other people (we don't do everything from scratch ourselves all the time). As a result, now we've fetched the 'gpt-2' code, we now need to fetch all the _other_ code that that, in turn, relies on. This is what the next line does. However, there's an important _gotcha_ here. Once we've fetched it, our notebook won't know how to use it, unless we restart it first (think of it like rebooting your phone after a software update). **After you execute the next cell, you'll see a message explaining this, and a button that says _'Restart runtime'_. Click this button, and then return to the first cell of the notebook (Underneath 'The code starts here') and run all the cells up to this point again.** You won't see the message or the button again and can continue the second time around.

In [0]:
%%capture --no-display
# Install the requirements. If you're prompted to 'restart runtime' then re-run this notebook after :-)
!pip install -r requirements.txt

Now, we're all ready to get on with the business of training our model! First we need to grab the _pre-trained_ GPT-2 model which we'll use as a base for our fine-tuning. This is the one which gives us the basics of the English language to start from. Do that by **running the following two cells**.

In [0]:
%%capture
# This cell makes sure the code in the cells below doesn't spew out loads of confusing debugging messages, so we can concentrate
# on looking at the output of the model. It's not really that interesting - just execute it and move on to the next one which is
# where the real work happens :-)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [0]:
%%capture
# Download the pre-trained 117-million-parameter GPT-2 model for fine-tuning
!python download_model.py 117M

Now we've downloaded this model, we can immediately start generating text from it, before we start fine-tuning it! This is actually quite an instructive exercise, as it gives us an idea of what sort of text it was trained on. **Run the cell below to see a few examples (it might take a minute or two)**. What do you see?

In [0]:
import generate_unconditional_samples as unconditional
unconditional.print_samples(model_name='117M', nsamples=5, length=144)

It's understandable English, but it's a bit all over the place, subject-wise! This is kinda to be expected - the training set was gathered from all over the anglophone Internet, so it's pretty diverse. Let's see if we can make gpt-2 generate something a bit more thematically coherent. To do this, let's grab your own _training set_ from Google drive. First we need to give this notebook permission to access your files from Google (Don't worry, only you will be able to see this). **Run the cell below then follow the instructions that it prints out: It'll ask you to visit another page (the link will open in a new tab) and sign into google, then copy a code from that tab into the text field that will appear below. Once you've done that, hit enter to continue.**

In [0]:
%%capture --no-display
# Connect to google Drive to get our data. Follow the instructions that are printed out below!
from google.colab import drive
drive.mount('/content/drive')

Now, we'll copy your training file across from Google drive! This is where we use the line you wrote above. You see where it says `training_file_name` below? that refers to the line of code you wrote at the start, and tells us where to find your data. To copy it, **execute the cell below**:

In [0]:
%%capture --no-stderr
# Copy the training text file from google drive into our working directory
from shutil import copyfile
copyfile("/content/drive/My Drive/%s" % training_file_name, "fine_tuning_training_set.txt")

Now, we need to convert this file into a format the computer can use more easily. Machine learning models work on numbers, so we need to convert your text into a _numeric representation_ - some format that uses numbers but preserves the meaning of the text. In our case, each word in the English language is assigned a number, and each sentence then becomes a sequence of numbers like `[1, 2, 3, 1 , 5 , 52, 103, 2, 4]`, or whatever. We don't need to know the details though, as those gpt-2 folks have done all the hard work for us. **Simply run the cell below and we'll get on with the real business of making talking robots**.

In [0]:
# Transform our training text into a format we can train from
%%capture
!PYTHONPATH=src python encode.py fine_tuning_training_set.txt fine_tuning_training_set.npz

Right, we've got all our ingredients ready - all the code we need, the existing _pre-trained_ GPT-2 model, and our own _training set_ in a format that the computer can understand, now **we're ready to train our model!** Again, all it takes is a single command to start this process. However... the tricky part is knowing when to stop it. The computer will carry on forever, trying to get better and better at mimicking the text we give it in the training set, but (a) we don't want to wait forever (we've got robots we need to talk to!), and (b) if it gets *too good*, it'll just end up repeating the example phrases we gave it in our training set, which would be a bit boring (and a bit pointless). 

This is a problem that machine learning researchers and engineers call _over-fitting_ - when a model gets so accurate that it just repeats the training data it was given, and can't _generalise_ to new examples. There are a lot of strategies for avoiding it, but we're going to do something simple. When we run the line below, the computer will print out messages below it that indicate its progress as it trains the model. This contains two numbers, called `loss` and `avg`. The **loss** is the amount of error at each training step - the difference between the output of the model and the training set we gave it. We want this to get nice and low, but not *too* low (if this difference is too small, it's likely we've overfit). The *loss* can jump up and down over the course of training, so instead we print out the **avg**, which is the _average loss_ over the last few training steps. This smoothes out those jumps, and should be a nice steadily descending number. What we'll do is keep an eye on this, and when it stops decreasing for a while, we'll tell the computer to stop the training process. 

You'll see some other information about the training process too, and occasionally it'll print out examples of sentances its generated for you to inspect. You can scroll up and down to see how these change throughout the process. Are they getting closer to what you expected to see?

**Run the cell below, and keep an eye on the output. It might take a minute or two to start printing out the loss figures (you'll see a few other lines of text first). When the `avg` figure gets down to below 2 and stays there for 10 or so steps, then click the little 'stop' button to the left of the cell (where 'play' was before) - it might take a second or two to respond (only click it once, even if it looks like it's not doing anything)**. This will stop the cell and save the model (the last like you see in the output should begin with  "`Saving checkpoint`") and we can move onto the next step. If you don't see "`Saving checkpoint`", re-run the cell before moving on!

**If your model doesn't get down to 2, or it takes ages, you can stop early by doing the same thing, no problem.** '2' is a bit of an arbitrary figure I've chosen here based on my own observations training this stuff - there's nothing particularly special about it. (Arguably, you should probably run it for a lot longer, but my aim with this notebook is to help people understand this stuff quickly and with fairly low effort, and I didn't want you to have to wait for a model to train overnight to get to that point). One aspect of the 'art' of doing this stuff for real is learning when to recognise you've reached that 'goldilocks point' of 'not-too-little, not-too-much' error, so don't worry if your model doesn't get to this point. In practice, if the average loss hasn't dropped for a while, then you've probably gone as far as you can go without overfitting, and it's time to look at what the model will output.

In [0]:
%%capture --no-stdout --no-display
!PYTHONPATH=src python train.py --dataset fine_tuning_training_set.npz --run_name fine_tuning

Loading checkpoint checkpoint/fine_tuning/model-98
Loading dataset...
100% 1/1 [00:00<00:00, 154.73it/s]
dataset has 110249 tokens
Training...
[99 | 29.61] loss=3.00 avg=3.00
Generating samples...
interrupted
Saving checkpoint/fine_tuning/model-100


Now our model is trained, we can start using it to generate text! Let's repeat the process we did before, to see a few samples frorm the new fine-tuned model by **running the two cells below**:

In [0]:
%%capture
# Copy the fine-tuned model and the data it needs into a new folder so we can refer to it below:
!mkdir -p models/fine_tuning
!cp -r checkpoint/fine_tuning/* models/fine_tuning
!cp models/117M/encoder.json models/fine_tuning
!cp models/117M/hparams.json models/fine_tuning
!cp models/117M/vocab.bpe models/fine_tuning

In [0]:
# Generate some samples. If you're feeling eagle-eyed, you might notice that there's a line in the cell
# we used before starting with `import` which we don't have here. This is because we only need to do this
# once for the whole notebook - it just brings the code we need to generate samples into the notebook, 
# and then we can re-use it as many times as we like.
unconditional.print_samples(model_name='fine_tuning', nsamples=5, length=144)

One interesting thing to note, is that the GPT-2 model gives us some control over how it generates these samples, in the form of _parameters_. We can adjust these parameters to change the text it gives us back in various ways. Here's a list of a few of them:

 - **`nsamples`**: This stands for 'number of samples', and does exactly what it says on the tin - it controls the number of samples that we get back from the model. Generating more samples will take the computer longer though!
 - **`length`**: The length of the sample in characters. Longer samples also take longer to generate.

 The next two are a bit more abstract, but still very useful:
 - **`temperature`** - This controls the randomness of the _word combinations_ in the sentances generated. A higher _temperature_ will often choose to put words together that don't normally go together so often, while a lower _temperature_ will play it safe, and stick to word combinations it's confident will go together. If your examples are a bit safe and boring, try raising the _temperature_ a bit, and if they're absolutely incoherent and all-over-the-place, bring it down a little. We've started at a _temperature_ of `1.0` which is normally a sensible default.
 - **`top_p`** - This controls the diversity of the _vocabularity_ that we pick words from. A high _top_p_ will use a very wide vocabulary, while a small _top_p_ will only use the most common words. If your model is repetitive, raise this a bit, and if it's a bit incoherent, or returns words that are total nonsense, lower it a bit. We've started at a temperature of `0.9` which tends to give good results.

**You can tweak these parameters with the sliders below on the right, and re-run the two cells below to see the changes** (notice how the numbers on the left change as you move the sliders!). Once you're happy with the responses you're getting, we'll move on to the final part of the notebook!


In [0]:
#@title Text generation parameters

nsamples = 2 #@param {type:"slider", min:1, max:10, step:1}
length  = 50 #@param {type:"slider", min:10, max: 300, step:5}
temperature =  1.0 #@param {type:"slider", min:0, max:3, step:0.1}
top_p = 0.9 #@param { type: "slider", min: 0, max:1, step: 0.001}




In [0]:
unconditional.print_samples(
    model_name='fine_tuning', 
    nsamples=nsamples, 
    length=length,
    temperature=temperature,
    top_p=top_p
)

Great stuff! So far, we've downloaded a pre-trained GPT-2 text generation model, fine-tuned it on our own data, and used it to generate some synthetic text, as well as learning about the text generation parameters and tweaking them to (hopefully) produce better examples, which isn't bad at all for a single line of code of actual programming. For our final task, we're going to make our talking robot respond to the others, by making a simple chat-bot!

We do this using a slightly different text-generation technique to the one we used before. Our previous samples were generated by a process called _unconditional sampling_ - they're just chosen at random from the entire space of possiblities texts the model can generate. Another technique we can use is called _conditional sampling_ - we give the model the beginning of the text, and ask it to chose some text which is likely to follow it. This is the principle that 'autocomplete' works on, and it can produce some quite interesting results. **You can try it out be editing the 'prompt' text in the form below to the right, and running the two cells below.**


In [0]:
#@title Initial text to complete
text_to_complete = "Once upon a time there was " #@param{type:"string"}

In [0]:
import generate_conditional_samples as conditional
print("%s..." % text_to_complete)
conditional.print_samples(
    text_to_complete, 
    model_name='fine_tuning',
    nsamples=nsamples, 
    length=length, 
    temperature=temperature,
    top_p=top_p
)

We can use this functionality to create a 'chat-bot', of sorts. **Run the cell below, and type in some text when prompted**! Can you have an interesting conversation with your bot?

In [0]:
with conditional.setup_model(
    model_name='fine_tuning',
    nsamples=1, 
    length=length, 
    temperature=temperature,
    top_p=top_p
) as arguments:
  print("Say hello to your new robot!")
  while True:
    text = input(">> ")
    for (_, result) in conditional.iterate_samples(text, *arguments):
      print("\t<< " + result.replace("\n", "\n\t<< "))

